In [1]:
import numpy as np
from StringIO import StringIO
import string

In [57]:
data = np.genfromtxt('movie_metadata_copy_Kay_edited.csv', dtype=None, delimiter=',', autostrip=True)
colNames = data[0,:]
movieData = np.delete(data, (0), axis=0)
numMovies = movieData.shape[0]
print numMovies 

5041


In [64]:
genres = {'Action', 'Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy',
          'Film-Noir','Game-Show','History','Horror','Music','Musical','Mystery','News','Reality-TV','Romance','Sci-Fi',
          'Short','Sport','Thriller','War','Western'}

moviesByGenreBucket = {}

for genre in genres:
    moviesByGenreBucket[genre] = []
    

In [65]:
for movie in movieData:
    localGenres = string.split(movie[10], "|")
    for genre in localGenres:
        #print "adding movieID = %s to genre bucket %s" % (movie[0], genre)
        moviesByGenreBucket[genre].append(movie[0])

In [66]:
for key in moviesByGenreBucket.keys():
    print "%s = %s" % (key, len(moviesByGenreBucket[key]))


Sci-Fi = 616
Crime = 889
Romance = 1107
Animation = 242
Music = 214
Comedy = 1872
War = 212
Horror = 564
Film-Noir = 6
Western = 97
News = 3
Reality-TV = 2
Thriller = 1409
Adventure = 922
Mystery = 499
Short = 5
Drama = 2592
Action = 1153
Documentary = 121
Musical = 132
History = 206
Family = 546
Fantasy = 610
Game-Show = 1
Sport = 182
Biography = 293


In [61]:
print moviesByGenreBucket['Game-Show']

['2144']


In [63]:
movieData[2143]

array(['2144', 'Color', '', '5', '60', '', '', '', '98', '',
       'Game-Show|Reality-TV|Romance', 'Chris Harrison',
       '"""The Bachelor            """', '4398', '98', '', '0',
       'bachelor|seeking love|single guy|tv host|women rivals for man',
       'http://www.imdb.com/title/tt0313038/?ref_=fn_tt_tt_1', '33',
       'English', 'USA', '', '3000000', '', '', '2.9', '', '141'], 
      dtype='|S149')